In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:
def prepare_prophet_df(df, date_col='date', target_col='close'):
    """
    Rename columns to fit Prophet's expected format: 'ds' and 'y'.
    """
    logger.info("Preparing data for Prophet...")
    prophet_df = df[[date_col, target_col]].rename(columns={date_col: 'ds', target_col: 'y'})
    prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])
    return prophet_df


In [ ]:
def train_prophet_model(df, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False):
    """
    Fit a Prophet model to the given dataframe.
    """
    logger.info("Training Prophet model...")
    model = Prophet(
        yearly_seasonality=yearly_seasonality,
        weekly_seasonality=weekly_seasonality,
        daily_seasonality=daily_seasonality
    )
    model.fit(df)
    return model


In [ ]:
def make_forecast(model, periods=30, freq='D'):
    """
    Generate future dataframe and make predictions.
    """
    logger.info(f"Making forecast for {periods} periods...")
    future = model.make_future_dataframe(periods=periods, freq=freq)
    forecast = model.predict(future)
    return forecast


In [ ]:
def evaluate_forecast(actual_df, forecast_df):
    """
    Compare actual vs predicted values and return metrics.
    """
    logger.info("Evaluating forecast...")
    merged = pd.merge(actual_df, forecast_df[['ds', 'yhat']], on='ds', how='inner')
    mae = mean_absolute_error(merged['y'], merged['yhat'])
    rmse = mean_squared_error(merged['y'], merged['yhat'], squared=False)
    return {'MAE': mae, 'RMSE': rmse}


In [ ]:
def plot_forecast(model, forecast, title='Prophet Forecast'):
    """
    Plot forecast with confidence intervals.
    """
    logger.info("Plotting forecast...")
    fig = model.plot(forecast)
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.tight_layout()
    plt.show()


In [ ]:
df = pd.read_csv("Data/processed/feature_engineered_stock_data.csv")

# Prepare for Prophet: rename columns
df_prophet = df.reset_index()[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])


In [ ]:
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
model.fit(df_prophet)


In [ ]:
future = model.make_future_dataframe(periods=60, freq='B')
forecast = model.predict(future)


In [ ]:
merged = pd.merge(
    df_prophet[['ds', 'y']],
    forecast[['ds', 'yhat']],
    on='ds',
    how='inner'
)


merged.rename(columns={'ds': 'Date', 'y': 'Actual', 'yhat': 'Forecast'}, inplace=True)


In [ ]:
merged.to_csv("prophet_forecast.csv", index=False)
print("Prophet forecast saved to 'data/processed/prophet_forecast.csv'")
